In [1]:
from __future__ import division, unicode_literals

In [2]:
import os 
from pprint import pprint

In [3]:
import numpy as np
import decimal

In [13]:
from pathlib2 import Path
import os

In [4]:
import streamm

In this getting started example we will calculate the electronic properties of methane with NWChem and the structural properties with LAMMPS

Now let's create project and resource to keep track of our work

In [5]:
Methane_example = streamm.Project('Methane_example')

And a resource object to keep track of where our files are 

In [14]:
need_files = ['local_res.json','remote_res.json']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run resource_example.ipynb")
        os.system("jupyter nbconvert --to python  resource_example.ipynb")
        os.system("python resource_example.py")

Load resources from resources example 

In [7]:
res_local = streamm.Resource('local')

The calc resource can be changed to local or remote host resouce 

In [15]:
res_calc = streamm.Resource('res_calc')

In [16]:
res_local.import_json()
res_calc.import_json()

In [17]:
pprint(res_calc.properties)

{u'allocation': u'orgopv',
 u'e-mail': u'my_username@nrel.gov',
 u'exe_command': u'qsub ',
 u'feature': u'24core',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': u'short',
 u'scratch': u'/scratch/my_username',
 u'walltime': 4}


In [18]:
Methane_example.set_resource(res_calc)

In [19]:
res_calc.make_dir()

Create .xyz file using a molecular viewer, such as Avogadro (https://avogadro.cc/) or explicitely as in the structure.ipynb example.

In [20]:
ME = streamm.Buildingblock('methane')

In [21]:
ME.read_xyz()

In [22]:
print(ME.write_xyz_str())

 5 
 methane 
     C       0.00000000       0.00000000       0.00000000 
     H       0.69282032       0.69282032       0.69282032 
     H      -0.69282032      -0.69282032       0.69282032 
     H      -0.69282032       0.69282032      -0.69282032 
     H       0.69282032      -0.69282032      -0.69282032 



Looks good let's set up some calculations

In [23]:
calc_i = streamm.Gaussian('g_methane_HF')

In [24]:
Methane_example.add_calc(calc_i) # Add it to the project 

In [25]:
calc_i.strucC = ME               # set the strucC to the structure container 

In [26]:
print calc_i.tag

g_methane_HF


Let's use the remote resource to run this calculation 

In [27]:
calc_i.set_resource(res_calc)

* home : directory is the root directory for the calculation/project
* template : directory of template input and run files to be modified to run the calculation 
* materials : directory to store structure files (.xyz) 
* launch : directory to temporarly store files before they are copied to the remote resouce 
* scratch : directory to run the calculation
* storage : directory to store completed calculation data

In [28]:
pprint(calc_i.dir)

{u'home': u'/Users/tkemper/Development/streamm-tools/examples',
 u'launch': u'/Users/tkemper/Development/streamm-tools/examples/scratch/g_methane_HF/',
 u'materials': u'/Users/tkemper/Development/streamm-tools/examples/materials',
 u'scratch': u'/scratch/my_username/g_methane_HF/',
 u'scripts': u'/Users/tkemper/Development/streamm-tools/examples/scripts',
 u'storage': u'/mss/users/my_username/g_methane_HF/',
 u'templates': u'/Users/tkemper/Development/streamm-tools/examples/../templates/'}


In [29]:
calc_i.make_dir()

In [30]:
print calc_i.dir['launch']

/Users/tkemper/Development/streamm-tools/examples/scratch/g_methane_HF/


In [31]:
os.chdir(calc_i.dir['launch'])

In [33]:
file_type = 'templates'
file_key = 'run'
file_name = "gaussian_remote.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [34]:
file_type = 'templates'
file_key = 'com'
file_name = "gaussian.com"
from_dirkey = 'templates'
to_dirkey = 'launch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [35]:
calc_i.load_str('templates','com')        
calc_i.load_str('templates','run')

In [36]:
calc_i.properties['commands'] = 'HF/3-21G SP'
calc_i.properties['charge'] = 0
calc_i.properties['spin_mult'] = 1
calc_i.properties['coord'] = calc_i.strucC.write_coord()

In [37]:
calc_i.replacewrite_prop('com','input','com','%s.com'%(calc_i.tag))

In [38]:
calc_i.properties['input_com'] = calc_i.files['input']['com']
calc_i.replacewrite_prop('run','scripts','run','%s.pbs'%(calc_i.tag))

In [39]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(calc_i.tag)
calc_i.add_file(file_type,file_key,file_name)

In [40]:
file_type = 'output'
file_key = 'fchk'
file_name = "%s.fchk"%(calc_i.tag)
calc_i.add_file(file_type,file_key,file_name)

Save details in .json files 

In [43]:
os.chdir(calc_i.dir['home'])
Methane_example.export_json()

{u'calculations': {'g_methane_HF': u'gaussian'},
 u'meta': {'date': '2017-11-14T18:26:05.614922',
  'software': u'streamm_proj',
  'status': 'written'},
 u'resources': ['peregrine', 'local']}

In [44]:
os.chdir(calc_i.dir['launch'])

In [45]:
calc_i.push()

In [46]:
calc_i.run()

Cool. While that is in the queue let's setup some more jobs

Let's also run a NWChem calculation

In [47]:
nwchem_i = streamm.NWChem('nw_methane_HF')

In [48]:
Methane_example.add_calc(nwchem_i)

In [49]:
nwchem_i.strucC = ME

In [50]:
print nwchem_i.tag

nw_methane_HF


In [51]:
nwchem_i.set_resource(res_calc)

In [52]:
pprint(nwchem_i.properties['scratch'])

u'/scratch/my_username/nw_methane_HF/'


In [53]:
nwchem_i.make_dir()

In [54]:
print nwchem_i.dir['launch']

/Users/tkemper/Development/streamm-tools/examples/scratch/nw_methane_HF/


In [55]:
os.chdir(nwchem_i.dir['launch'])

In [56]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem_remote.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [57]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [58]:
nwchem_i.load_str('templates','nw')        
nwchem_i.load_str('templates','run')

In [60]:
nwchem_i.properties['basis'] = '6-31g'
nwchem_i.properties['method'] = 'UHF'
nwchem_i.properties['charge'] = 0
nwchem_i.properties['spin_mult'] = 1
nwchem_i.properties['task'] = 'SCF '
nwchem_i.properties['coord'] = nwchem_i.strucC.write_coord()

In [61]:
pprint(nwchem_i.properties)

{u'allocation': u'orgopv',
 u'basis': u'6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 u'coord': u'     C       0.00000000       0.00000000       0.00000000 \n     H       0.69282032       0.69282032       0.69282032 \n     H      -0.69282032      -0.69282032       0.69282032 \n     H      -0.69282032       0.69282032      -0.69282032 \n     H       0.69282032      -0.69282032      -0.69282032 \n',
 u'e-mail': u'my_username@nrel.gov',
 u'exe_command': u'qsub ',
 u'feature': u'24core',
 u'finish_str': u'Total times  cpu:',
 u'maxiter': 100,
 u'method': u'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': u'short',
 u'scratch': u'/scratch/my_username/nw_methane_HF/',
 u'spin_mult': 1,
 u'task': u'SCF ',
 'uncompress': 'tar -xzf ',
 u'walltime': 4}


In [62]:
nwchem_i.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_i.tag))

In [63]:
nwchem_i.properties['input_nw'] = nwchem_i.files['input']['nw']
nwchem_i.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_i.tag))

In [64]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_i.tag)
nwchem_i.add_file(file_type,file_key,file_name)

Save details in .json files 

In [67]:
os.chdir(nwchem_i.dir['home'])
Methane_example.export_json()

{u'calculations': {'g_methane_HF': u'gaussian', 'nw_methane_HF': u'nwchem'},
 u'meta': {'date': '2017-11-14T18:26:05.614922',
  'software': u'streamm_proj',
  'status': 'written'},
 u'resources': ['peregrine', 'local']}

In [68]:
os.chdir(nwchem_i.dir['launch'])

In [69]:
nwchem_i.push()

In [70]:
nwchem_i.run()

Okay we have a couple calculations now, so let's check their status

In [71]:
Methane_example.check()

Calculation g_methane_HF has status written
Calculation nw_methane_HF has status written


In [72]:
nwchem_i.analysis()

Running analysis on  nw_methane_HF.log


In [73]:
print nwchem_i.properties['alpha_energies']

[]


In [74]:
print nwchem_i.properties['N_alpha_occ']

0


In [75]:
Methane_example.store()

In [76]:
Methane_example.pull()

/Users/tkemper/Development/streamm-tools/examples/scratch/nw_methane_HF
Calculation g_methane_HF has status written
/Users/tkemper/Development/streamm-tools/examples/scratch/nw_methane_HF
Calculation nw_methane_HF has status written


In [77]:
os.chdir(nwchem_i.dir['home'])
Methane_example.export_json()

{u'calculations': {'g_methane_HF': u'gaussian', 'nw_methane_HF': u'nwchem'},
 u'meta': {'date': '2017-11-14T18:26:05.614922',
  'software': u'streamm_proj',
  'status': 'written'},
 u'resources': ['peregrine', 'local']}

Neat!

Now let's optimize the structure and calculate the ESP charges

In [78]:
nwchem_opt = streamm.NWChem('nw_methane_OPT')

In [79]:
Methane_example.add_calc(nwchem_opt)

In [80]:
nwchem_opt.strucC = ME

In [81]:
print nwchem_opt.tag

nw_methane_OPT


In [82]:
nwchem_opt.set_resource(res_calc)

In [83]:
pprint(nwchem_opt.properties['scratch'])

u'/scratch/my_username/nw_methane_OPT/'


In [84]:
nwchem_opt.make_dir()

In [85]:
print nwchem_opt.dir['launch']

/Users/tkemper/Development/streamm-tools/examples/scratch/nw_methane_OPT/


In [86]:
os.chdir(nwchem_opt.dir['launch'])

In [87]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem_remote.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_opt.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [88]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_opt.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [89]:
nwchem_opt.load_str('templates','nw')        
nwchem_opt.load_str('templates','run')

In [90]:
nwchem_opt.properties['basis'] = '6-31g'
nwchem_opt.properties['method'] = 'UHF'
nwchem_opt.properties['charge'] = 0
nwchem_opt.properties['spin_mult'] = 1
nwchem_opt.properties['task'] = 'SCF optimize'
nwchem_opt.properties['coord'] = nwchem_opt.strucC.write_coord()

In [91]:
pprint(nwchem_opt.properties)

{u'allocation': u'orgopv',
 u'basis': u'6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 u'coord': u'     C       0.00000000       0.00000000       0.00000000 \n     H       0.69282032       0.69282032       0.69282032 \n     H      -0.69282032      -0.69282032       0.69282032 \n     H      -0.69282032       0.69282032      -0.69282032 \n     H       0.69282032      -0.69282032      -0.69282032 \n',
 u'e-mail': u'my_username@nrel.gov',
 u'exe_command': u'qsub ',
 u'feature': u'24core',
 u'finish_str': u'Total times  cpu:',
 u'maxiter': 100,
 u'method': u'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': u'short',
 u'scratch': u'/scratch/my_username/nw_methane_OPT/',
 u'spin_mult': 1,
 u'task': u'SCF optimize',
 'uncompress': 'tar -xzf ',
 u'walltime': 4}


In [92]:
nwchem_opt.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_opt.tag))

In [93]:
nwchem_opt.properties['input_nw'] = nwchem_opt.files['input']['nw']
nwchem_opt.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_opt.tag))

In [94]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_opt.tag)
nwchem_opt.add_file(file_type,file_key,file_name)

Save details in .json files 

In [95]:
os.chdir(nwchem_opt.dir['home'])
Methane_example.export_json()

{u'calculations': {'g_methane_HF': u'gaussian',
  'nw_methane_HF': u'nwchem',
  'nw_methane_OPT': u'nwchem'},
 u'meta': {'date': '2017-11-14T18:26:05.614922',
  'software': u'streamm_proj',
  'status': 'written'},
 u'resources': ['peregrine', 'local']}

In [96]:
os.chdir(nwchem_opt.dir['launch'])

In [97]:
nwchem_opt.push()

In [98]:
nwchem_opt.run()

In [99]:
nwchem_opt.check()

In [100]:
print nwchem_opt.meta

{'date': '2017-11-14T18:36:28.223641', 'status': 'written', 'resource': 'peregrine', 'software': u'nwchem'}


In [101]:
Methane_example.check()

Calculation nw_methane_OPT has status written
Calculation g_methane_HF has status written
Calculation nw_methane_HF has status written


In [102]:
Methane_example.store()

In [103]:
Methane_example.pull()

/Users/tkemper/Development/streamm-tools/examples/scratch/nw_methane_OPT
Calculation nw_methane_OPT has status written
/Users/tkemper/Development/streamm-tools/examples/scratch/nw_methane_OPT
Calculation g_methane_HF has status written
/Users/tkemper/Development/streamm-tools/examples/scratch/nw_methane_OPT
Calculation nw_methane_HF has status written


In [104]:
nwchem_opt.analysis()

Running analysis on  nw_methane_OPT.log


In [105]:
print nwchem_opt.strucC.write_xyz_str()

 5 
 methane 
     C       0.00000000       0.00000000       0.00000000 
     H       0.69282032       0.69282032       0.69282032 
     H      -0.69282032      -0.69282032       0.69282032 
     H      -0.69282032       0.69282032      -0.69282032 
     H       0.69282032      -0.69282032      -0.69282032 



In [106]:
os.chdir(nwchem_opt.dir['materials'])

In [107]:
nwchem_opt.strucC.tag = '{}_{}'.format(nwchem_opt.strucC.tag,nwchem_opt.tag)

In [108]:
nwchem_opt.strucC.write_xyz()

In [109]:
nwchem_esp = streamm.NWChem('nw_methane_ESP')

In [110]:
ME_OPT = streamm.Buildingblock('methane_nw_methane_OPT')

In [111]:
ME_OPT.read_xyz()

In [112]:
print(ME.write_xyz_str())

 5 
 methane_nw_methane_OPT 
     C       0.00000000       0.00000000       0.00000000 
     H       0.69282032       0.69282032       0.69282032 
     H      -0.69282032      -0.69282032       0.69282032 
     H      -0.69282032       0.69282032      -0.69282032 
     H       0.69282032      -0.69282032      -0.69282032 



In [113]:
Methane_example.add_calc(nwchem_esp)

In [114]:
nwchem_esp.strucC = ME_OPT

In [115]:
print nwchem_esp.tag

nw_methane_ESP


In [116]:
nwchem_esp.set_resource(res_calc)

In [117]:
pprint(nwchem_esp.properties['scratch'])

u'/scratch/my_username/nw_methane_ESP/'


In [118]:
nwchem_esp.make_dir()

In [119]:
print nwchem_esp.dir['launch']

/Users/tkemper/Development/streamm-tools/examples/scratch/nw_methane_ESP/


In [120]:
os.chdir(nwchem_esp.dir['launch'])

In [121]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem_remote.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_esp.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [122]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_esp.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [123]:
nwchem_esp.load_str('templates','nw')        
nwchem_esp.load_str('templates','run')

In [124]:
nwchem_esp.properties['basis'] = '6-31g'
nwchem_esp.properties['method'] = 'UHF'
nwchem_esp.properties['charge'] = 0
nwchem_esp.properties['spin_mult'] = 1
nwchem_esp.properties['task'] = 'esp'
nwchem_esp.properties['coord'] = nwchem_esp.strucC.write_coord()

In [125]:
pprint(nwchem_esp.properties)

{u'allocation': u'orgopv',
 u'basis': u'6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 u'coord': u'     C       0.00000000       0.00000000       0.00000000 \n     H       0.69282032       0.69282032       0.69282032 \n     H      -0.69282032      -0.69282032       0.69282032 \n     H      -0.69282032       0.69282032      -0.69282032 \n     H       0.69282032      -0.69282032      -0.69282032 \n',
 u'e-mail': u'my_username@nrel.gov',
 u'exe_command': u'qsub ',
 u'feature': u'24core',
 u'finish_str': u'Total times  cpu:',
 u'maxiter': 100,
 u'method': u'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': u'short',
 u'scratch': u'/scratch/my_username/nw_methane_ESP/',
 u'spin_mult': 1,
 u'task': u'esp',
 'uncompress': 'tar -xzf ',
 u'walltime': 4}


In [126]:
nwchem_esp.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_esp.tag))

In [127]:
nwchem_esp.properties['input_nw'] = nwchem_esp.files['input']['nw']
nwchem_esp.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_esp.tag))

In [128]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_esp.tag)
nwchem_esp.add_file(file_type,file_key,file_name)

Save details in .json files 

In [130]:
os.chdir(nwchem_esp.dir['home'])
Methane_example.export_json()

{u'calculations': {'g_methane_HF': u'gaussian',
  'nw_methane_ESP': u'nwchem',
  'nw_methane_HF': u'nwchem',
  'nw_methane_OPT': u'nwchem'},
 u'meta': {'date': '2017-11-14T18:26:05.614922',
  'software': u'streamm_proj',
  'status': 'written'},
 u'resources': ['peregrine', 'local']}

In [131]:
os.chdir(nwchem_esp.dir['launch'])

In [132]:
nwchem_esp.push()

In [133]:
nwchem_esp.run()

In [134]:
print nwchem_esp.tag,nwchem_esp.files['output']

nw_methane_ESP {u'log': u'nw_methane_ESP.log'}


In [135]:
nwchem_esp.check()

In [136]:
print nwchem_esp.meta

{'date': '2017-11-14T18:37:33.454610', 'status': 'written', 'resource': 'peregrine', 'software': u'nwchem'}


In [137]:
Methane_example.check()

Calculation nw_methane_OPT has status written
Calculation g_methane_HF has status written
Calculation nw_methane_HF has status written
Calculation nw_methane_ESP has status written


In [138]:
Methane_example.store()

In [139]:
Methane_example.pull()

/Users/tkemper/Development/streamm-tools/examples/scratch/nw_methane_ESP
Calculation nw_methane_OPT has status written
/Users/tkemper/Development/streamm-tools/examples/scratch/nw_methane_ESP
Calculation g_methane_HF has status written
/Users/tkemper/Development/streamm-tools/examples/scratch/nw_methane_ESP
Calculation nw_methane_HF has status written
/Users/tkemper/Development/streamm-tools/examples/scratch/nw_methane_ESP
Calculation nw_methane_ESP has status written


In [140]:
nwchem_opt.analysis()

Running analysis on  nw_methane_OPT.log


Now we have an optimized molecular geometery and ESP charges